In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta 




In [2]:
#data with feature engineering
fields = ['Date', 'Open','High','Low','Close','Volume','RSI','MACD','Ichimoku_Span_A','Ichimoku_Span_B','ATR',
         'BB_High_Indicator','BB_Low_Indicator']
MSFT_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/MSFT.csv", usecols=fields)
AAPL_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL.csv", usecols=fields)
AMZN_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AMZN.csv", usecols=fields)



In [15]:
MSFT_df_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_monthly.csv", index_col=0)
AAPL_df_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_monthly.csv", index_col=0)
AMZN_df_monthly =pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_monthly.csv", index_col=0)



In [17]:
MSFT_df_monthly.columns = ["Open","High","Low","Close","Volume"]

#drop the last row of each dataframe so the entries match
#MSFT_df_monthly.drop(MSFT_df_monthly.tail(1).index,inplace=True)

AAPL_df_monthly.columns =["Open","High","Low","Close","Volume"]
#AAPL_df_monthly.drop(AAPL_df_monthly.tail(1).index,inplace=True)

AMZN_df_monthly.columns = ["Open","High","Low","Close","Volume"]
#AMZN_df_monthly.drop(AMZN_df_monthly.tail(1).index,inplace=True)

In [19]:
#reversed monthly Dataframe 
reversed_df_MSFT = MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN = AMZN_df_monthly.iloc[::-1]

In [21]:
def labelling_multiclass(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    dataframe = dataframe.apply(pd.to_numeric)
    #introduce the absolute price change as a variable
    dataframe['ATR'] = ta.volatility.average_true_range(dataframe.High, dataframe.Low, dataframe.Close, n=14, 
                                                        fillna=True)
    
    #calculate the percentage change of ATR between months 
    dataframe['ATR_%_change'] = abs(dataframe['ATR'].pct_change())
    
    #calculate the average ATR pct change
    average_ATR_pct_change = dataframe['ATR_%_change'].mean()
    
    
    
    empty_dict = dict()
    prev_close =0
    counter = 0
    
    #print (dataframe.head(5))
    
    for index,cols in dataframe.iterrows():
        
        counter +=1
        
        current_close = cols['Close']
        ATR_price_change = cols['ATR_%_change']
        
        if (float(prev_close) <= float(current_close)) and (float(ATR_price_change) <= float(average_ATR_pct_change)):
            
            #signals an increase in price between last month and next month
            
            empty_dict[index] = 1
        # 2 represents an increase that was strong greater than the average ATR   
        elif (float(prev_close) <= float(current_close)) and (float(ATR_price_change) >= float(average_ATR_pct_change)):
            empty_dict[index] = 2
        
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) <= float(average_ATR_pct_change))):
            empty_dict[index]=0
            
        #3 represents a decrease in the price that had a larger ATR    
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) >= float(average_ATR_pct_change))):
            empty_dict[index] = 3
        
        else:
            empty_dict[index]= 0
            

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [23]:
#ADD LABELS TO THE REVERESED DATAFRAMES USING THE LABELLING FUNCTION 
MSFT_labels = labelling_multiclass(reversed_df_MSFT)
AAPL_labels = labelling_multiclass(reversed_df_AAPL)
AMZN_labels = labelling_multiclass(reversed_df_AMZN)

In [24]:
#Add the labels to the reversed DataFrames
reversed_df_MSFT['Labels'] = MSFT_labels.values()
reversed_df_AAPL['Labels'] = AAPL_labels.values()
reversed_df_AMZN['Labels']= AMZN_labels.values()



/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [26]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            empty_dict[month_key]= temp_df.values.flatten()
            count +=1
        else:
            pass
       
    return empty_dict
    
#get dictionary of the period end dates with the 30days previous features minus date as a flattened array of 360 entries
#reshape 30,12 will give the original dataframe config
MSFT_30days_flattened= get_ndays_monthly_flattened(reversed_df_MSFT,MSFT_df,30)
AAPL_30days_flattened = get_ndays_monthly_flattened(reversed_df_AAPL,AAPL_df,30)
AMZN_30days_flattened = get_ndays_monthly_flattened(reversed_df_AMZN,AMZN_df,30)

In [27]:
#assign the labels to pandas dataframe objects and give label title
MSFT_labels_df = pd.Series(MSFT_labels).to_frame("Label")
AAPL_labels_df= pd.Series(AAPL_labels).to_frame("Label")
AMZN_labels_df= pd.Series(AMZN_labels).to_frame("Label")

In [30]:
#convert the flattened dictionaries to dataframes 
flattened_df_MSFT = pd.DataFrame.from_dict(MSFT_30days_flattened).T
flattened_df_AAPL = pd.DataFrame.from_dict(AAPL_30days_flattened).T
flattened_df_AMZN = pd.DataFrame.from_dict(AMZN_30days_flattened).T

In [31]:
#merge the dataframes on the index 'Date', and ensure that only the dates present in the flattened dataframe are carried over

MSFT_flat_df = flattened_df_MSFT.merge(MSFT_labels_df, left_index=True, right_index=True)
AAPL_flat_df = flattened_df_AAPL.merge(AAPL_labels_df, left_index=True, right_index=True)
AMZN_flat_df = flattened_df_AMZN.merge(AMZN_labels_df, left_index=True, right_index=True)

In [43]:
#split the data into training and test and unfortunately we miss a couple months in the middle for some strange reason?
MSFT_flat_df_98_15 = MSFT_flat_df[:204]
MSFT_flat_df_15_19 = MSFT_flat_df[206:]

In [48]:
#split the data approximately 80% to 20%
AAPL_flat_df_98_15 = AAPL_flat_df[:204]
AAPL_flat_df_15_19 = AAPL_flat_df[206:]

In [50]:
#split the data approximately into %80 to %20 
AMZN_flat_df_98_15 = AMZN_flat_df[:204]
AMZN_flat_df_15_19 = AMZN_flat_df[206:]

In [ ]:
export_98_15 = MSFT_flat_df_98_15.to_csv("multiclass_MSFT_98_15.csv")
export_15_19 = MSFT_flat_df_15_19.to_csv("multiclass_MSFT_15_19.csv")
export_98_15 = AAPL_flat_df_98_15.to_csv("multiclass_AAPL_98_15.csv")
export_15_19 = AAPL_flat_df_15_19.to_csv("multiclass_AAPL_15_19.csv")
export_98_15 = AMZN_flat_df_98_15.to_csv("multiclass_AMZN_98_15.csv")
export_15_19 = AMZN_fl.to_csv("multiclass_AMZN_15_19.csv")
